<a href="https://colab.research.google.com/github/alex6095/SanctiMoly/blob/main/BTTtoMars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치

In [ ]:
#@title
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# %cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab190912.sh

!pip install konlpy transformers Korpora sentencepiece

     |████████████████████████████████| 19.4 MB 8.1 MB/s 
     |████████████████████████████████| 3.1 MB 48.6 MB/s 
     |████████████████████████████████| 57 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 
     |████████████████████████████████| 448 kB 50.4 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
     |████████████████████████████████| 61 kB 622 kB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 895 kB 61.9 MB/s 
     |████████████████████████████████| 3.3 MB 50.0 MB/s 
     |████████████████████████████████| 96 kB 7.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beaut

# 기본 패키지 로드

In [ ]:
import os, sys
from google.colab import drive

from easydict import EasyDict

import math
import random
import time
import re

from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.utils.data as tud
import torch.nn as nn
import torch.optim as optim
import gc


from Korpora import Korpora
from Korpora import ModuNewsKorpus
import transformers
from transformers import BertModel, DistilBertModel

import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab
from collections import Counter


from cProfile import Profile
from pstats import Stats

# torch.backends.cudnn.benchmark = True

# 기본 설정

In [ ]:
# ?: https://stackoverflow.com/questions/60002128/failed-to-assign-a-dict-with-key-of-type-int-to-easydict
C = EasyDict()

C.gdrive = {}
# GDrive 마운트 설정
C.gdrive.mount_path = '/gdrive'
C.gdrive.root = '/cs492i'

C.corpus = {}
# 말뭉치 작은거 사용
C.corpus.use_smallset = True

C.dataset = {}
# 데이터셋 나누기 비율
C.dataset.split_train = 0.8
# 데이터셋 배치 크기
C.dataset.batch_size = 64 # 64


C.torch = {}
# Torch Seed 고정 (사용 안하면 None)
C.torch.manual_seed = None
# 가능하면 GPU 사용
C.torch.use_gpu = True



# 설정 적용 및 잡일

In [ ]:
if C.torch.manual_seed is not None:
    torch.manual_seed(C.torch.manual_seed)
    torch.cuda.manual_seed(C.torch.manual_seed)
    print('Using torch manual seed:', C.torch.manual_seed)

device = torch.device("cuda" if torch.cuda.is_available() and C.torch.use_gpu else "cpu")
print('Using device:', device.type)

Using device: cuda


In [ ]:
# 드라이브 로드
drive.mount(C.gdrive.mount_path)
root = '%s/My Drive%s' % (C.gdrive.mount_path, C.gdrive.root)


Mounted at /gdrive


In [ ]:
%cd $root
%pwd

/gdrive/.shortcut-targets-by-id/1uCYxyZyGrrk_QuM7N7v-PYCAmnJ6SKlW/cs492i


'/gdrive/.shortcut-targets-by-id/1uCYxyZyGrrk_QuM7N7v-PYCAmnJ6SKlW/cs492i'

# 드라이브에서 파일 임포트

In [ ]:
# load .py files from google drive
from tokenization_kobert import KoBertTokenizer

# 유틸 정의

In [ ]:
class NewsDateDataset(tud.Dataset):
    """News with Date dataset."""

    def __init__(self, corpus, news_transform=None, date_transform=None):
        """
        Args:
            corpus (Corpus): Korpora corpus with news object.
            news_transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.corpus = corpus
        self.news_transform = news_transform
        self.date_transform = date_transform

    def __len__(self):
        return len(self.corpus.train)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Option apply
        paragraph = self.corpus.train[idx].paragraph
        if self.news_transform is not None:
            paragraph = self.news_transform(paragraph)
        
        date = (int(corpus.train[idx].date[0:4]), int(corpus.train[idx].date[4:6]))
        if self.date_transform is not None:
            date = self.date_transform(date)
        
        return paragraph, date


In [ ]:
# tokenizer = tk_hanbert.HanBertTokenizer.from_pretrained('HanBert-54kN-torch')
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
"""
tokenizer.tokenize(문자열)
tokenizer.convert_tokens_to_ids(문자열)
tokenizer.covert_tokens_to_string(문자열)
"""

# bert_model = BertModel.from_pretrained('monologg/kobert')

distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')


Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/108M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/distilkobert were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# One-Hot Encoding tensot

one_hot = F.one_hot(torch.arange(120))
one_hot = torch.as_tensor(one_hot, dtype = torch.float32)
print(one_hot)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])


In [ ]:
class Compose(object):
    """Composes several transforms together. Copied from torchvision.transforms.Compose"""

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img):
        for t in self.transforms:
            img = t(img)
        return img

    def __repr__(self):
        format_string = self.__class__.__name__ + '('
        for t in self.transforms:
            format_string += '\n'
            format_string += '    {0}'.format(t)
        format_string += '\n)'
        return format_string


# 엄청 느릴듯
class RegexSubstitution(object):
    """Regex substitution class for transform"""

    def __init__(self, regex, sub=''):
        if isinstance(regex, re.Pattern):
            self.regex = regex
        else:
            self.regex = re.compile(regex)
        self.sub = sub
    
    def __call__(self, target):
        if isinstance(target, list):
            return [ self.regex.sub(self.sub, self.regex.sub(self.sub, string)) for string in target ]
        else:
            return self.regex.sub(self.sub, self.regex.sub(self.sub, target))

class SquashParagraph(object):
    """Squash paragraph"""

    def __call__(self, sample):
        return '\n'.join(sample)

class ExcludeTitle(object):
    """Exclude title"""

    def __call__(self, sample):
        return sample[1:]

class PreprocessString(object):
    """ [CLS] ? [SEP] """

    def __call__(self, sample):
        return "[CLS] " + sample + " [SEP]"

class PackToFloat(object):
    """ year, month -> float """

    def __call__(self, date):
        year, month = date
        return (12 * (year - 2009) + month - 1) / 120
        # month 1~12 to 0~11

class PackToInt(object):
    """ year, month -> int """

    def __call__(self, date):
        year, month = date
        return (12 * (year - 2009) + month - 1)
        # month 1~12 to 0~11

class OneHotEnc(object):
    """ year, month -> torch.tensor(120) """

    def __call__(self, date):
        year, month = date
  
        return one_hot[(12 * (year - 2009) + month - 1)]

def bert_tokenizer_old(sent, MAX_LEN=3000):
    input_ids, attention_masks, token_type_ids = [], [], []
    for text in sent:
        encoded_dict = tokenizer.encode_plus(
            text=text, 
            add_special_tokens=True, 
            max_length=MAX_LEN, 
            padding='max_length', 
            return_attention_mask=True,
            truncation=True)
        input_id = encoded_dict['input_ids']
        attention_mask = encoded_dict['attention_mask']
        token_type_id = encoded_dict['token_type_ids']
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)

    return input_ids, attention_masks, token_type_ids

def bert_tokenizer(sent, MAX_LEN=512):
    encoded_dict = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=sent,
        add_special_tokens=True, 
        max_length=MAX_LEN, 
        padding='max_length', 
        return_attention_mask=True,
        truncation=True)
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']

    return input_id, attention_mask, token_type_ids

def distilbert_tokenizer(sent, MAX_LEN=512):
    encoded_dict = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=sent,
        add_special_tokens=True, 
        max_length=MAX_LEN, 
        padding='max_length', 
        return_attention_mask=True,
        truncation=True)
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']

    return input_id, attention_mask

def text_pipeline(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

# https://androidkt.com/create-dataloader-with-collate_fn-for-variable-length-input-in-pytorch/
def collate_batch_old(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=tokenizer.pad_token_id)
    return text_list.to(device), label_list.to(device)

def collate_batch_pack(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        text_list.append(_text)
    # as_tensor로 복사 방지
    label_list = torch.as_tensor(label_list, dtype=torch.float32)
    # int64 -> float32 아닌가?
    # print(label_list)
    btr_a, btr_b, btr_c = bert_tokenizer(text_list)
    btr_a = torch.as_tensor(btr_a)
    btr_b = torch.as_tensor(btr_b)
    btr_c = torch.as_tensor(btr_c)
    # print(btr_a.requires_grad)
    return btr_a.to(device), btr_b.to(device), btr_c.to(device), label_list.to(device)

def collate_batch_OH(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        text_list.append(_text)
    # as_tensor로 복사 방지
    label_tensor = torch.stack(label_list)
    # int64 -> float32 아닌가?
    # print(label_list)
    btr_a, btr_b = distilbert_tokenizer(text_list)
    btr_a = torch.as_tensor(btr_a)
    btr_b = torch.as_tensor(btr_b)
    
    # print(btr_a.requires_grad)
    return btr_a, btr_b, label_tensor

def collate_batch(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        text_list.append(_text)
        
    # as_tensor로 복사 방지
    label_list = torch.as_tensor(label_list, dtype=torch.float32)
    # int64 -> float32 아닌가?
    # print(label_list)
    btr_a, btr_b = distilbert_tokenizer(text_list)
    btr_a = torch.as_tensor(btr_a)
    btr_b = torch.as_tensor(btr_b)
    # print(btr_a.requires_grad)
    return btr_a, btr_b, label_list

# 데이터셋 로드

In [ ]:
# 말뭉치 로드
corpus_force_load = False
if (not ('corpus' in globals())) or (corpus is None) or corpus_force_load:
    corpus = None
    if C.corpus.use_smallset:
        corpus = ModuNewsKorpus(root_dir = root + '/smallnews', force_download=False, load_light=False)
    else:
        corpus = ModuNewsKorpus(root_dir = root + '/news', force_download=False, load_light=False)
else:
    print('Corpus already loaded')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    모두의 말뭉치는 문화체육관광부 산하 국립국어원에서 제공하는 말뭉치로
    총 13 개의 말뭉치로 이뤄져 있습니다.

    해당 말뭉치를 이용하기 위해서는 국립국어원 홈페이지에 가셔서 "회원가입 > 말뭉치 신청 > 승인"의
    과정을 거치셔야 합니다.

    https://corpus.korean.go.kr/#none

    모두의 말뭉치는 승인 후 다운로드 가능 기간 및 횟수 (3회) 에 제한이 있습니다.

    로그인 기능 및 Korpora 패키지에서의 다운로드 기능을 제공하려 하였지만,
    국립국어원에서 위의 이유로 이에 대한 기능은 제공이 불가함을 확인하였습니다.

    Korpora==0.2.0 에서는 "개별 말뭉치 신청 > 승인"이 완료되었다고 가정,
    로컬에 다운로드 된 말뭉치를 손쉽게 로딩하는 기능만 제공합니다

    (Korpora 개발진 lovit@github, ratsgo@github)

    # License
    모두의 말뭉치의 모든 저작권은 `문화체육관광부 국립국어원
    (National Institute of Korean Language)` 에 귀속됩니다.

    소유권을 포함한 전문은 다음의 주소에서 확인하실 수 있습니다.

    https://corpus.korean.go.kr/boards/termsInfo.do

    제13조 (소유권)
    ① 누리집이 제공하는 서비스, 그에 필요한 소프트웨어, 이미지, 마크, 로고, 디자인

Loading ModuNews: 100%|██████████| 10/10 [00:28<00:00,  2.89s/it]


In [ ]:
nds_dataset = NewsDateDataset(corpus, news_transform=Compose([ExcludeTitle(),
                                                              SquashParagraph(),
                                                              RegexSubstitution(r'\([^()]+\)|[<>\'"△▲□■]'),
                                                              # PreprocessString()
                                                              ]),
                              date_transform=OneHotEnc() )

print("Total news:", len(nds_dataset))

n_train = math.floor(C.dataset.split_train * len(nds_dataset))
n_val = len(nds_dataset) - n_train
print("Dataset split to (train, val) = (%d, %d)" % (n_train, n_val))

train_dataset, valid_dataset = tud.random_split(nds_dataset, [n_train, n_val])

train_loader = tud.DataLoader(train_dataset,
                              batch_size=C.dataset.batch_size,
                              shuffle=True,
                              drop_last=False,
                              collate_fn=collate_batch_OH,
                              )
valid_loader = tud.DataLoader(valid_dataset,
                              batch_size=C.dataset.batch_size,
                              shuffle=False,
                              drop_last=False,
                              collate_fn=collate_batch_OH,
                              )


Total news: 115080
Dataset split to (train, val) = (92064, 23016)


In [ ]:
# TODO: sorting batch?
# 2009~2018 : (12*(year-2009)+month)/12*10
# max_len = -1
# for i_batch, sample_batched in tqdm(enumerate(train_loader)):
#     # print(i_batch, sample_batched)
#     max_len = sample_batched[0].size(0) if max_len < sample_batched[0].size(0) else max_len
#     print(sample_batched[2])
# print(max_len)

In [ ]:
input = torch.tensor([0.0, 1.0, 2.0], requires_grad=True)
w = torch.tensor([[1.0], [1.0], [1.0]], requires_grad=False)
(w*input).requires_grad

True

# 모델

In [ ]:
class SanctiMoly(nn.Module):
    """ Holy Moly News BERT """

    def __init__(self, freeze_bert = True):
        super(SanctiMoly, self).__init__()
        self.encoder = distilbert_model
        # FC-BN-Tanh
        self.linear = nn.Sequential(nn.Linear(768, 1024),
                                    nn.BatchNorm1d(1024),
                                    nn.Tanh(),
                                    nn.Dropout(),
                                    nn.Linear(1024, 768),
                                    nn.BatchNorm1d(768),
                                    nn.Tanh(),
                                    nn.Dropout(),
                                    nn.Linear(768, 120)
                                    )
        # self.softmax = nn.LogSoftmax(dim=-1)

        if freeze_bert == True:
            for param in self.encoder.parameters():
                param.requires_grad = False
        else:
            for param in self.encoder.parameters():
                param.requires_grad = True

            
    def forward(self, input_ids, attention_mask): # Token_type_ids는 상관믕므
        enc_o = self.encoder(input_ids, attention_mask)
        
        output = self.linear(enc_o.last_hidden_state[:, 0, :])
        # print(output.shape)
        return output

# nn.Seq?

In [ ]:
model = SanctiMoly(freeze_bert=False).to(device)


# for param in model.parameters():
#     print(param.requires_grad)

# print(model.named_parameters)
# for name, param in model.named_parameters():
#     if name in ['linear.0.weight', 'linear.0.bias', 'linear.3.weight', 'linear.3.bias', 'linear.6.weight', 'linear.6.bias']:
#         param.requires_grad = True
#         print(name)
#     else:
#         param.requires_grad = False

# bert_model.to(device)
# print(next(bert_model.parameters()).device)

In [ ]:
print(torch.cuda.memory_allocated()/1024/1024)
!nvidia-smi

113.51123046875
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Sat Dec  4 01:14:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    32W / 250W |   1247MiB / 16280MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+---

# 학습

In [ ]:
criterion = None
def i2ym(fl):
    return (fl // 12 + 2009, fl % 12 + 1)
def save_model(model, mode="last"):
    result_dir = root+'/Result'
    os.makedirs(result_dir, exist_ok=True)
    torch.save(model.state_dict(),  result_dir+f'/{type(model).__name__}_BTT_{mode}.ckpt')
    
def run_epoch(epoch, model, optimizer, is_train=True, dataloader=None):
    total_loss = 0
    n_total = 0
    n_correct = 0
    # batch_size = C.dataset.batch_size
    if dataloader is None:
        dataloader = train_loader if is_train else valid_loader
    if is_train:
        model.train()
    else:
        model.eval()
    for batch in enumerate(tqdm(dataloader)):

        i_batch, sample_batched = batch
        s_a, s_b, sampled_label = sample_batched
        
        pred = None
        if is_train:
            pred = model(s_a.to(device), s_b.to(device))
        else:
            with torch.no_grad():
                pred = model(s_a.to(device), s_b.to(device))
        # print(pred, sampled_label)
        loss = criterion(pred, sampled_label.to(device))

        #print(loss.requires_grad)
        #loss = torch.autograd.Variable(loss, requires_grad = True)
        n_targets = sampled_label.size(0)
        n_total += n_targets
        total_loss += float(loss)

        copied_pred = pred.cpu()
        # One_hot
        b_correct = (copied_pred.argmax(-1) == sampled_label.argmax(-1)).long().sum().item()
        n_correct += b_correct

        # Int
        # 모델 output 복사후 사용 -> GPU 메모리 아끼기

        # b_correct = (torch.floor(pred.cpu()) == torch.floor(sampled_label.cpu())).long().sum().item()
        # n_correct += b_correct

        if i_batch%50 == 0:
            # print(torch.cuda.memory_allocated()/1024/1024, torch.cuda.memory_reserved()/1024/1024)
            print("Pred:",i2ym(copied_pred.argmax(-1)[0].item()),"Label:", i2ym(sampled_label.argmax(-1)[0].item()))
            print('[Batch {}]'.format(i_batch), 'Train' if is_train else 'Valid', 
            "Loss", float(loss) / n_targets, "Accuracy", b_correct * 100 / n_targets, "%")
        if is_train:
            optimizer.zero_grad()
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
            optimizer.step()
    total_loss /= n_total
    accu = n_correct * 100 / n_total
    print("Epoch", epoch, 'Train' if is_train else 'Valid', 
          "Loss", total_loss, "Accuracy", accu, "%")
    return total_loss

def run_experiment(model):
    global criterion
    # criterion = nn.MSELoss(size_average=None, reduce=None, reduction='mean')
    criterion = nn.CrossEntropyLoss()
    
    # 학습시킬 파라미터만 넣자 -> Freeze 경우

    optimizer = optim.AdamW(filter(lambda p: p.requires_grad==True, model.parameters()), lr=3e-5) # 0.001, 0.1, 3e-5
  
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
            factor=0.25, patience=1, threshold=0.0001, threshold_mode='rel', 
            cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    best_val_loss = np.inf

    for epoch in tqdm(range(10)):
        run_epoch(epoch, model, optimizer, is_train=True)
        with torch.no_grad():
            val_loss = run_epoch(epoch, model, None, is_train=False)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_model(model, 'best')
        save_model(model)

        scheduler.step(val_loss)
    

In [ ]:
run_experiment(model)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1439 [00:00<?, ?it/s]

Pred: (2017, 11) Label: (2017, 8)
[Batch 0] Train Loss 0.07849445194005966 Accuracy 0.0 %
Pred: (2011, 6) Label: (2009, 10)
[Batch 50] Train Loss 0.07667411863803864 Accuracy 0.0 %
Pred: (2011, 11) Label: (2014, 3)
[Batch 100] Train Loss 0.07498852908611298 Accuracy 3.125 %
Pred: (2012, 3) Label: (2012, 4)
[Batch 150] Train Loss 0.07435370981693268 Accuracy 1.5625 %
Pred: (2012, 8) Label: (2014, 5)
[Batch 200] Train Loss 0.07385364919900894 Accuracy 1.5625 %
Pred: (2017, 3) Label: (2009, 3)
[Batch 250] Train Loss 0.0766872987151146 Accuracy 1.5625 %
Pred: (2009, 8) Label: (2015, 9)
[Batch 300] Train Loss 0.07537025958299637 Accuracy 3.125 %
Pred: (2015, 8) Label: (2015, 3)
[Batch 350] Train Loss 0.07354585826396942 Accuracy 6.25 %
Pred: (2009, 1) Label: (2014, 2)
[Batch 400] Train Loss 0.07590329647064209 Accuracy 3.125 %
Pred: (2009, 1) Label: (2010, 5)
[Batch 450] Train Loss 0.07510446012020111 Accuracy 3.125 %
Pred: (2015, 3) Label: (2013, 10)
[Batch 500] Train Loss 0.07123903930187

  0%|          | 0/360 [00:00<?, ?it/s]

Pred: (2014, 5) Label: (2014, 5)
[Batch 0] Valid Loss 0.06565694510936737 Accuracy 9.375 %
Pred: (2009, 9) Label: (2010, 2)
[Batch 50] Valid Loss 0.06916430592536926 Accuracy 6.25 %
Pred: (2009, 9) Label: (2010, 4)
[Batch 100] Valid Loss 0.06688189506530762 Accuracy 4.6875 %
Pred: (2009, 9) Label: (2011, 7)
[Batch 150] Valid Loss 0.06654439866542816 Accuracy 1.5625 %
Pred: (2015, 4) Label: (2017, 10)
[Batch 200] Valid Loss 0.0680341050028801 Accuracy 7.8125 %
Pred: (2018, 9) Label: (2014, 2)
[Batch 250] Valid Loss 0.06791071593761444 Accuracy 6.25 %
Pred: (2018, 3) Label: (2011, 8)
[Batch 300] Valid Loss 0.0670166015625 Accuracy 9.375 %
Pred: (2009, 2) Label: (2009, 2)
[Batch 350] Valid Loss 0.06730079650878906 Accuracy 12.5 %
Epoch 0 Valid Loss 0.06779371724594456 Accuracy 5.122523461939521 %


...

Pred: (2010, 11) Label: (2010, 6)
[Batch 0] Train Loss 0.052833542227745056 Accuracy 15.625 %
Pred: (2009, 12) Label: (2009, 11)
[Batch 50] Train Loss 0.046462010592222214 Accuracy 18.75 %
Pred: (2009, 8) Label: (2009, 2)
[Batch 100] Train Loss 0.04704933241009712 Accuracy 21.875 %
Pred: (2013, 10) Label: (2013, 10)
[Batch 150] Train Loss 0.0509011372923851 Accuracy 14.0625 %
Pred: (2018, 2) Label: (2017, 2)
[Batch 200] Train Loss 0.05084693804383278 Accuracy 18.75 %
Pred: (2014, 7) Label: (2014, 8)
[Batch 250] Train Loss 0.04473239928483963 Accuracy 29.6875 %
Pred: (2012, 6) Label: (2010, 11)
[Batch 300] Train Loss 0.051541898399591446 Accuracy 12.5 %
Pred: (2018, 6) Label: (2014, 7)
[Batch 350] Train Loss 0.045491740107536316 Accuracy 25.0 %
Pred: (2013, 1) Label: (2013, 1)
[Batch 400] Train Loss 0.045778512954711914 Accuracy 26.5625 %
Pred: (2014, 4) Label: (2014, 5)
[Batch 450] Train Loss 0.05298583582043648 Accuracy 20.3125 %
Pred: (2009, 1) Label: (2009, 1)
[Batch 500] Train Loss

  0%|          | 0/360 [00:00<?, ?it/s]

Pred: (2014, 5) Label: (2014, 5)
[Batch 0] Valid Loss 0.049374449998140335 Accuracy 29.6875 %
Pred: (2010, 5) Label: (2010, 2)
[Batch 50] Valid Loss 0.05904387682676315 Accuracy 14.0625 %
Pred: (2010, 11) Label: (2010, 4)
[Batch 100] Valid Loss 0.0515083409845829 Accuracy 18.75 %
Pred: (2010, 8) Label: (2011, 7)
[Batch 150] Valid Loss 0.0551590621471405 Accuracy 14.0625 %
Pred: (2018, 10) Label: (2017, 10)
[Batch 200] Valid Loss 0.04998940974473953 Accuracy 15.625 %
Pred: (2014, 12) Label: (2014, 2)
[Batch 250] Valid Loss 0.05339960753917694 Accuracy 18.75 %
Pred: (2013, 1) Label: (2011, 8)
[Batch 300] Valid Loss 0.052753470838069916 Accuracy 17.1875 %
Pred: (2009, 9) Label: (2009, 2)
[Batch 350] Valid Loss 0.0556836798787117 Accuracy 20.3125 %
Epoch 7 Valid Loss 0.05256641915907742 Accuracy 18.30031282586027 %


  0%|          | 0/1439 [00:00<?, ?it/s]

Pred: (2013, 5) Label: (2013, 11)
[Batch 0] Train Loss 0.04766037315130234 Accuracy 18.75 %
Pred: (2011, 4) Label: (2011, 4)
[Batch 50] Train Loss 0.052377909421920776 Accuracy 18.75 %
Pred: (2018, 4) Label: (2017, 4)
[Batch 100] Train Loss 0.0450420044362545 Accuracy 21.875 %
Pred: (2009, 4) Label: (2009, 2)
[Batch 150] Train Loss 0.04604366049170494 Accuracy 14.0625 %
Pred: (2010, 6) Label: (2011, 2)
[Batch 200] Train Loss 0.04595038294792175 Accuracy 23.4375 %
Pred: (2010, 11) Label: (2010, 11)
[Batch 250] Train Loss 0.048561640083789825 Accuracy 18.75 %
Pred: (2017, 7) Label: (2018, 9)
[Batch 300] Train Loss 0.039681896567344666 Accuracy 35.9375 %
Pred: (2018, 2) Label: (2018, 2)
[Batch 350] Train Loss 0.04989242181181908 Accuracy 21.875 %
Pred: (2009, 2) Label: (2009, 3)
[Batch 400] Train Loss 0.04847484454512596 Accuracy 21.875 %
Pred: (2016, 7) Label: (2015, 7)
[Batch 450] Train Loss 0.04490651935338974 Accuracy 21.875 %
Pred: (2016, 11) Label: (2016, 11)
[Batch 500] Train Loss 

In [ ]:
print(torch.cuda.memory_allocated()/1024/1024)
!nvidia-smi

In [ ]:
# model.eval()

# def i2ym(fl):
#     return (fl // 12 + 2009, fl % 12 + 1)

# with torch.no_grad():
#     for batch in enumerate(tqdm(valid_loader)):
#         i_batch, sample_batched = batch
#         s_a, s_b, sampled_label = sample_batched

#         pred = model(s_a.to(device), s_b.to(device))
        
#         total = 0
#         count=0
#         for i in range(C.dataset.batch_size):
#             print(i, ": expected", i2ym(sampled_label[i].item()), "| got", i2ym(pred[i].item()), "L1 dist", abs(sampled_label[i].item()-pred[i].item()))
#             total += abs(sampled_label[i].item()-pred[i].item())
#             if abs(sampled_label[i].item()-pred[i].item()) <= 6:
#                 count+=1
#         print(count*100/C.dataset.batch_size, total/C.dataset.batch_size)
#         break


# Playground

완벽한 신문기사  
34081

In [ ]:
# ridx = random.randint(0, len(corpus.train))
# # print(corpus.train[ridx].title)
# print("News #%d\n" % ridx)
# print(corpus.train[ridx].paragraph)
# print('==============================================')

# removal_regex = r'\([^()]+\)|[<>\'"△▲□■]'
# compiled_regex = re.compile(removal_regex)

# def preprocess(string):
#     return compiled_regex.sub('', compiled_regex.sub('', string))

# print(preprocess(corpus.train[ridx].paragraph))